In [2]:
import implicit
import scipy
#from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import psycopg2 as postgres
import pandas.io.sql as sqlio
from engine import engine
conn = engine.raw_connection()

In [6]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [232]:
sql_ratings = "select user_id, game_id, round(avg(rating)) rating from core.ratings group by user_id, game_id"
ratings = sqlio.read_sql(sql_ratings, engine)

In [231]:
ratings

,user_id,game_id,rating
0,1,403,8.0
1,1,648,3.0
2,1,780,8.0
3,1,815,8.0
4,1,909,7.0
...,...,...,...
260666,19559,5175,10.0
260667,19559,6952,8.0
260668,19560,1919,4.0
260669,19560,10661,6.0


In [53]:
wide_ratings=ratings.pivot(index='game_id', columns='user_id', values='rating').fillna(0)

In [54]:
wide_ratings

user_id,1,2,3,4,5,6,7,8,9,10,...,19551,19552,19553,19554,19555,19556,19557,19558,19559,19560
game_id,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
game_user_matrix=scipy.sparse.csr_matrix(wide_ratings)

In [221]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(game_user_matrix)

In [222]:
user_game_matrix = game_user_matrix.T.tocsr()

In [225]:
model.recommend_all(user_game_matrix)

array([[3437, 8525, 2006, ...,  497, 8291, 4306],
       [4464, 6842, 7976, ...,  498, 2463, 1127],
       [ 493, 2008, 7525, ...,  502, 4464, 4282],
       ...,
       [2445, 1109, 2448, ..., 8291,  796, 7819],
       [3004, 3006, 5816, ..., 3313, 7527, 3848],
       [ 731, 6842, 7872, ..., 2695, 4418, 2446]], dtype=int32)

In [226]:
recommendations=[]
for user in range(0, wide_ratings.shape[1]):
    recommendations.append(model.recommend(user, user_game_matrix))
recommendations

[[(3437, 1.0279105),
  (8525, 1.0257694),
  (2006, 1.0256791),
  (8462, 1.0153044),
  (6867, 0.9979271),
  (3848, 0.9957356),
  (626, 0.9574816),
  (497, 0.9503422),
  (8291, 0.9309636),
  (4306, 0.9038713)],
 [(4464, 0.21705912),
  (6842, 0.14917603),
  (7976, 0.14736839),
  (6414, 0.14625037),
  (6844, 0.14389315),
  (4108, 0.1420188),
  (8787, 0.13951921),
  (498, 0.13769102),
  (2463, 0.13439411),
  (1127, 0.13343485)],
 [(493, 1.0517287),
  (2008, 0.82363164),
  (7525, 0.82101023),
  (2006, 0.6732598),
  (676, 0.6621264),
  (2005, 0.6346302),
  (7524, 0.62310344),
  (502, 0.5463064),
  (4464, 0.5288572),
  (4282, 0.5134908)],
 [(7647, 0.5400132),
  (8290, 0.4994669),
  (2926, 0.45647207),
  (727, 0.40127695),
  (724, 0.38793135),
  (7800, 0.37886214),
  (733, 0.37755728),
  (7872, 0.34446567),
  (5318, 0.3418333),
  (5817, 0.3381334)],
 [(3217, 0.75699717),
  (8291, 0.6841024),
  (8687, 0.6360287),
  (8324, 0.6291448),
  (7860, 0.60665786),
  (8290, 0.6046978),
  (7878, 0.59540814

In [79]:
model.similar_users(1)

[(1, 2.1461422),
 (5226, 1.3970715),
 (5122, 1.3970715),
 (5159, 1.3970715),
 (5177, 1.3970714),
 (5071, 1.3970712),
 (5033, 1.397071),
 (5321, 1.3970635),
 (5147, 1.3970634),
 (5101, 1.3970634)]

In [111]:
games=wide_ratings.index.values.tolist()
games

[5,
 21,
 22,
 23,
 24,
 25,
 27,
 29,
 36,
 37,
 41,
 43,
 44,
 48,
 49,
 50,
 51,
 58,
 59,
 63,
 64,
 65,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 77,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 88,
 89,
 93,
 94,
 98,
 99,
 100,
 102,
 104,
 105,
 110,
 115,
 116,
 117,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 130,
 131,
 132,
 141,
 142,
 144,
 145,
 147,
 150,
 151,
 155,
 156,
 158,
 159,
 160,
 163,
 164,
 170,
 171,
 173,
 176,
 177,
 179,
 180,
 182,
 186,
 187,
 188,
 191,
 194,
 195,
 197,
 202,
 205,
 206,
 207,
 209,
 216,
 220,
 221,
 222,
 224,
 225,
 226,
 227,
 228,
 229,
 235,
 237,
 238,
 240,
 244,
 245,
 248,
 249,
 250,
 254,
 255,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 279,
 281,
 282,
 286,
 287,
 288,
 294,
 299,
 300,
 301,
 302,
 303,
 305,
 307,
 308,
 309,
 311,
 312,
 313,
 314,
 315,
 317,
 318,
 320,
 321,
 323,
 324,
 326,
 331,
 333,
 334,
 335,
 336,
 337,
 338,
 342,
 344,
 346,
 347,
 

In [227]:
simtest=model.similar_items(games.index(5282))
simgames=[]
for tup in simtest:
    simgames.append(games[tup[0]])
simgames
#TODO: use dictionary to map indices with ids!

[5282, 5281, 5283, 5279, 5278, 8322, 13029, 4320, 10157, 9674]

Very good results! Similar games of GTA V based on ratings are:  
GTA II, GTA IV, GTA San Andreas, GTA Vice City, ..., Read Dead Redemption, Skyrim


In [216]:
# sample user profile: GTA IV, GTA V and RDR 2
# game_ids 5279, 5282, 10158
sample_profile=[float(0)]*8951
sample_profile[games.index(5279)]=float(10)
sample_profile[games.index(5282)]=float(10)
sample_profile[games.index(10158)]=float(10)
sample_profile_csr=scipy.sparse.csr_matrix(sample_profile)
# sample user profile: Battlefield 1, Call of Duty: WWII and Tom Clancys Rainbow Six: Siege
# game_ids 1301, 2008, 13791
sample_profile_2=[float(0)]*8951
sample_profile_2[games.index(1301)]=float(10)
sample_profile_2[games.index(2008)]=float(10)
sample_profile_2[games.index(13791)]=float(10)
sample_profile_2_csr=scipy.sparse.csr_matrix(sample_profile_2)

In [228]:
rockstar_user_rec=model.recommend(0, sample_profile_csr, recalculate_user=True)
recgames=[]
for tup in rockstar_user_rec:
    recgames.append(games[tup[0]])
    #TODO: use dictionary to map indices with ids!
recgames

[5281, 5283, 10157, 5278, 5170, 7572, 13211, 5858, 13029, 6868]

good recommendations for user that liked GTA IV, GTA V and RDR 2:  
God of War, other GTAs, Horizon, L.A. Noire, Maffia, Spider-Man, Red Dead Redemption, Last of Us

In [229]:
rockstar_user_rec=model.recommend(0, sample_profile_2_csr, recalculate_user=True)
recgames=[]
for tup in rockstar_user_rec:
    recgames.append(games[tup[0]])
    #TODO: use dictionary to map indices with ids!
recgames

[1308, 11873, 13554, 1307, 2012, 13798, 1310, 4686, 7786, 13704]

recommending games for new users generally works, check if sample_profile_csr is in correct format...